In [10]:
from datetime import datetime, timedelta
from SQLiteFun import execution
from hapiclient import hapi

# check and see if data can be retrieved from HAPI server
def DataChecker(server, dataset, start, stop, parameters):    
    # Get data
    data, meta = hapi(server, dataset, parameters, start, stop)
    
    dataLen = len(data)
    print("The length of data is " + str(dataLen))
    
    # control loop
    dataFound = False
    while not dataFound:
        i = 0
        for entry in data:
            print(type(data[i][1]))
            
            # possible type mismatch when using boolean operator
            
            # place in a try/except to wait x seconds and try again if this fails
            # if 3 fails in a row -> no data
            
            # once data is found, end loop
            if data[i][1] != "":
                dataFound = True
                print("Data was successfully accessed")
                print("Example data looks like " + str(data[i][1]))
                break
            else:
                print("No data was found")
            print(data[i][1])
            i += 1
    return dataFound
    
# retrieve prodKeys associated with HAPI URLs
HapiStmt = """ SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 10"""
prodKeys = execution(HapiStmt)
print("The prodKeys are " + str(prodKeys))


server = 'https://cdaweb.gsfc.nasa.gov/hapi'

# iterate thru prodKeys
for prodKey in prodKeys:
    # control loop
    dataFound = False
    # list that holds all parameters for a given server
    paramNames =  []

    dataset = str(prodKey)

    # retrieve all parameters and start and stop date from the server
    meta = hapi(server,dataset)
    for k, v in meta.items():
        if k == "parameters":
            for params in v:
                for key, value in params.items():
                    if key == "name":
                        paramNames.append(value)
        elif k == "startDate":
            start = v
            date, sep, time = start.partition("T")
            time = time.replace("Z", "")
            dt_string = date + " " + time
            dt_obj = datetime.strptime(dt_string, "%Y-%m-%d %H:%M:%S")
            stop = dt_obj + timedelta(minutes=30)
            stop = str(stop)
            stop = stop.replace(" ", "T") + "Z"

    #print("Start time is " + start)
    #print("Stop time is " + stop)
    #print(paramNames)

    #start      = '1997-09-02T00:00:12Z'
    #stop       = '1997-09-02T00:30:12Z'
    #parameters = ",".join(paramNames[1:])
    #print(parameters)

    # iterate thru parameters in a server to see if any data can be pulled
    for parameters in paramNames[1:]:
        if not dataFound:
            print("Checking parameter " + str(parameters) + " in HAPI record with id " + dataset)
            dataFound = DataChecker(server, dataset, start, stop, parameters)


The prodKeys are ['JUNO_HELIO1DAY_POSITION', 'IA_K0_MFI', 'IA_K0_EPI', 'IA_K0_ENF', 'IA_OR_DEF', 'IA_K0_ICD', 'HELIOS1_COHO1HR_MERGED_MAG_PLASMA', 'HEL1_6SEC_NESSMAG', 'HELIOS1_HELIO1DAY_POSITION', 'HELIOS1_E6_1HOUR_PARTICLE_FLUX']
Checking parameter RAD_AU in HAPI record with id JUNO_HELIO1DAY_POSITION
The length of data is 1
<class 'numpy.float64'>
Data was successfully accessed
Example data looks like 1.0099999904632568
Checking parameter B_abs in HAPI record with id IA_K0_MFI
The length of data is 15
<class 'numpy.float64'>
Data was successfully accessed
Example data looks like 1608.0
Checking parameter Fe1 in HAPI record with id IA_K0_EPI
The length of data is 1
<class 'numpy.float64'>
Data was successfully accessed
Example data looks like -1e+31
Checking parameter E1 in HAPI record with id IA_K0_ENF
The length of data is 8
<class 'numpy.float64'>
Data was successfully accessed
Example data looks like 8.800000190734863
Checking parameter SC_pos_GSE in HAPI record with id IA_OR_DEF

/tmp/ipykernel_1935/846359233.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if data[i][1] != "":


The length of data is 2
<class 'numpy.int32'>
Data was successfully accessed
Example data looks like 1
Checking parameter RotationNumber in HAPI record with id HELIOS1_COHO1HR_MERGED_MAG_PLASMA


HAPIError: Failed to connect to: https://cdaweb.gsfc.nasa.gov/hapi/info?id=HELIOS1_COHO1HR_MERGED_MAG_PLASMA If problem persists, a contact email for the server may be listed at http://hapi-server.org/servers/